In [1]:
from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps

import orjson

In [2]:
import ray

In [3]:
ray.init()

2024-11-05 19:15:10,816	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.10.12
Ray version:,2.37.0
Dashboard:,http://127.0.0.1:8266


(bulk_insert pid=64464) (sqlite3.OperationalError) no such table: geometries
(bulk_insert pid=64464) [SQL: INSERT INTO geometries (name,geom) VALUES (?,GeomFromGeoJSON(?));]
(bulk_insert pid=64464) [parameters: [('tubule', '{"type":"Polygon","coordinates":[[[45837,20092],[45836,20093],[45835,20094],[45834,20095],[45833,20096],[45832,20096],[45831,20096],[45830,20096],[45 ... (12576 characters truncated) ... 20092],[45846,20092],[45845,20092],[45844,20092],[45843,20092],[45842,20092],[45841,20092],[45840,20092],[45839,20092],[45838,20092],[45837,20092]]]}'), ('lumen', '{"type":"Polygon","coordinates":[[[45823,20293],[45822,20294],[45821,20295],[45820,20294],[45819,20294],[45818,20295],[45817,20296],[45816,20296],[45 ... (1306 characters truncated) ... 20300],[45830,20299],[45829,20299],[45828,20298],[45827,20297],[45826,20296],[45826,20295],[45825,20294],[45825,20293],[45824,20293],[45823,20293]]]}'), ('lumen', '{"type":"Polygon","coordinates":[[[45848,20249],[45848,20250],[45848,20251]

In [4]:
ray.cluster_resources()

{'accelerator_type:G': 1.0,
 'node:__internal_head__': 1.0,
 'CPU': 16.0,
 'object_store_memory': 6236005171.0,
 'memory': 12472010343.0,
 'node:172.17.185.211': 1.0,
 'GPU': 1.0}

In [5]:
%%time
with open('13_266069_040_003 L02 PAS.json', 'r') as file:
#with open('/mnt/c/research/kidney/15_26609_024_045 L03 PAS.json', 'r') as file:
    # Load the JSON data into a Python dictionary
    data = orjson.loads(file.read())

import shapely
from shapely.geometry import shape

CPU times: user 11.3 s, sys: 1.66 s, total: 13 s
Wall time: 14.5 s


In [6]:
data[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[45837, 20092],
    [45836, 20093],
    [45835, 20094],
    [45834, 20095],
    [45833, 20096],
    [45832, 20096],
    [45831, 20096],
    [45830, 20096],
    [45829, 20096],
    [45828, 20096],
    [45827, 20096],
    [45826, 20096],
    [45825, 20096],
    [45824, 20096],
    [45823, 20096],
    [45822, 20096],
    [45821, 20096],
    [45820, 20097],
    [45819, 20098],
    [45818, 20099],
    [45817, 20100],
    [45816, 20100],
    [45815, 20100],
    [45814, 20100],
    [45813, 20100],
    [45812, 20101],
    [45811, 20102],
    [45810, 20103],
    [45809, 20104],
    [45808, 20104],
    [45807, 20104],
    [45806, 20104],
    [45805, 20104],
    [45804, 20105],
    [45803, 20106],
    [45802, 20107],
    [45801, 20108],
    [45800, 20109],
    [45799, 20110],
    [45798, 20111],
    [45797, 20112],
    [45796, 20113],
    [45795, 20114],
    [45794, 20115],
    [45793, 20116],
    [45792, 20117],
    [45791, 

In [7]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))

/tmp/ipykernel_63939/2237727833.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [8]:
# Connect to the Spatialite database
db_path = 'sqlite:////tmp/blog/spatialite_ray.db'
engine = create_engine(db_path, echo=False)  # Set echo=True to see SQL commands being executed

In [9]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')
    dbapi_connection.execute("PRAGMA timeout= 1000;")    

In [10]:
# Create the table
Base.metadata.create_all(engine)

In [11]:
from tqdm import tqdm

In [16]:
@ray.remote
def bulk_insert(polygons):
    engine = create_engine('sqlite:////tmp/blog/spatialite_ray.db')#,echo=True)
    
    try:
        # Initialize Spatialite extension
        @event.listens_for(engine, "connect")
        def connect(dbapi_connection, connection_record):
            dbapi_connection.enable_load_extension(True)
            dbapi_connection.execute('SELECT load_extension("mod_spatialite")')
            dbapi_connection.execute("PRAGMA busy_timeout= 10000000;")   
    
        with engine.connect() as conn:
            with conn.begin() as tran:
                res=conn.execute(text("INSERT INTO geometries (name,geom) VALUES (:name,GeomFromGeoJSON(:geom));"),polygons)
    except Exception as inst:
        print(inst)
        pass
    finally:
        engine.dispose() ##might be needed? --- yes needed

In [17]:
%%time
futures = [] 
for _ in range(12):
    batch_size=5_000
    polygons=[]
    
    for geojson in tqdm(data):
        name = geojson["properties"]["classification"]["name"]
        geometry = orjson.dumps(geojson["geometry"],).decode("utf-8")
    
        polygons.append({'name':name,'geom':geometry})
    
        
        if len(polygons) == batch_size:
            futures.append(bulk_insert.remote(polygons))
            polygons=[]
    
    if polygons:
        futures.append(bulk_insert.remote(polygons))
    
for f in tqdm(futures):
    ray.get(f)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 216/216 [05:14<00:00,  1.46s/it]

CPU times: user 15.3 s, sys: 9.55 s, total: 24.9 s
Wall time: 5min 36s


In [18]:
%%time
#lets make sure insert worked as expected
with  engine.connect() as conn:
    res=conn.execute(text("select count(geom) from geometries"))
    nresults=res.fetchall()
    print(nresults)

[(1063260,)]
CPU times: user 656 ms, sys: 5.16 s, total: 5.81 s
Wall time: 12.3 s


In [19]:
%%time
with  engine.connect() as conn:
    res=conn.execute(text("select AsGeoJSON(ST_centroid(geom)) from geometries limit 1000"))
    centroids=res.fetchall()

CPU times: user 27.6 ms, sys: 2.38 ms, total: 30 ms
Wall time: 27.1 ms


In [20]:
centroids[0:100]

[('{"type":"Point","coordinates":[45862.13292750414,20242.07223759453]}',),
 ('{"type":"Point","coordinates":[45812.26248864668,20306.06145927944]}',),
 ('{"type":"Point","coordinates":[45854.85213032581,20261.92932330827]}',),
 ('{"type":"Point","coordinates":[45901.874369386,20236.45970222714]}',),
 ('{"type":"Point","coordinates":[46006.16937669376,20198.66260162601]}',),
 ('{"type":"Point","coordinates":[45858.96278317151,20359.42055016181]}',),
 ('{"type":"Point","coordinates":[45792.37442014578,20346.95361166335]}',),
 ('{"type":"Point","coordinates":[45750.40160878947,20334.95173631548]}',),
 ('{"type":"Point","coordinates":[45954.05971937029,20293.65349075975]}',),
 ('{"type":"Point","coordinates":[45759.33590138675,20285.43374422187]}',),
 ('{"type":"Point","coordinates":[45982.3797250859,20206.2147766323]}',),
 ('{"type":"Point","coordinates":[45981.7408456021,20166.57002642506]}',),
 ('{"type":"Point","coordinates":[45803.70650616221,20158.2388936658]}',),
 ('{"type":"Point"

*** SIGTERM received at time=1730831207 on cpu 3 ***
PC: @     0x7fea7448ae2e  (unknown)  epoll_wait
    @     0x7fea743a7520  (unknown)  (unknown)
[2024-11-05 19:26:47,294 E 63939 63939] logging.cc:440: *** SIGTERM received at time=1730831207 on cpu 3 ***
[2024-11-05 19:26:47,294 E 63939 63939] logging.cc:440: PC: @     0x7fea7448ae2e  (unknown)  epoll_wait
[2024-11-05 19:26:47,294 E 63939 63939] logging.cc:440:     @     0x7fea743a7520  (unknown)  (unknown)
